In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

In [2]:
def fetch_and_save_stats(url, div_id=None, div_class=None, section_class=None):
    driver = webdriver.Chrome()
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
        if div_id:
            wait.until(EC.presence_of_element_located((By.ID, div_id)))
        elif div_class:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, div_class)))
        elif section_class:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, section_class)))
        
        page_source = driver.page_source
        soup = bs(page_source, 'html.parser')

        target_element = None
        if div_id:
            target_element = soup.find('div', {'id': div_id})
        elif div_class:
            target_element = soup.find('div', {'class': div_class})
        elif section_class:
            target_element = soup.find('section', {'class': section_class})
        else:
            target_element = soup.body  # Nếu không có thông số, lấy toàn bộ trang

        if not target_element:
            print("Error: Element with specified identifier not found.")
            return None

        content = []
        for elem in target_element.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'a']):
            text = elem.get_text(strip=True)
            if elem.name == 'a' and elem.has_attr('href'):
                text += f" ({elem['href']})"
            if text:
                content.append([text])

        df = pd.DataFrame(content, columns=["Text"])
        return df
    except Exception as e:
        print(f"Error occurred: {e}")
        return None
    finally:
        driver.quit()

In [3]:
def fetch_multiple_stats(urls):
    dataframes = {}
    for key, info in urls.items():
        df = fetch_and_save_stats(info["url"], div_id=info.get("div_id"), div_class=info.get("div_class"), section_class=info.get("section_class"))
        if df is not None:
            dataframes[key] = df
    return dataframes

In [4]:
urls = {
    "tuyen_sinh_ton_duc_thang": {
        "url": "https://tuyensinh.edu.vn/dai-hoc/truong-dh-ton-duc-thang-chi-danh-25-chi-tieu-xet-diem-thi-tot-nghiep-thpt/",
        "div_id": "content",
        "div_class": "entry-content single-page",
    },
    "tuyen_sinh_phenikaa": {
        "url": "https://tuyensinh.edu.vn/tin-tuyen-sinh/truong-dai-hoc-phenikaa-cong-bo-5-phuong-thuc-tuyen-sinh-dai-hoc-nam-2022/",
        "div_id": "content",
        "div_class": "entry-content single-page",
    },
}


In [5]:
dataframes = fetch_multiple_stats(urls)

for key, df in dataframes.items():
    print(f"\nDataFrame for {key}:")
    print(df)


DataFrame for tuyen_sinh_ton_duc_thang:
                                                 Text
0                              Đại học,Tin tuyển sinh
1         Đại học (https://tuyensinh.edu.vn/dai-hoc/)
2   Tin tuyển sinh (https://tuyensinh.edu.vn/tin-t...
3   Trường ĐH Tôn Đức Thắng chỉ dành 25% chỉ tiêu ...
4   Theo phương án tuyển sinh dự kiến, năm 2022 Tr...
5   Hôm nay 19.1, Trường ĐH Tôn Đức Thắng công bố ...
6   Trường ĐH Tôn Đức Thắng dự kiến tuyển 6.500 ch...
7   Trường dự kiến tuyển theo 5 phương thức, trong...
8   Phương thức 1là xét tuyển theo kết quả quá trì...
9   Trong đó, đợt 1 xét tuyển theo kết quả học tập...
10  Đợt 2 xét tuyển theo kết quả học tập 6 học kỳ ...
11  Đợt 3 xét tuyển theo kết quả học tập 6 học kỳ ...
12  Phương thức 2là xét tuyển theo kết quả thi tốt...
13  Phương thức 3là ưu tiên xét tuyển theo quy địn...
14  Trong đó, đối tượng 1 là thí sinh thuộc các tr...
15  Đối tượng 2, 3, 4, 5 xét tuyển vào chương trìn...
16  Phương thức 4là xét tuyển thẳng, ưu t

In [6]:
df_tonducthang = dataframes["tuyen_sinh_ton_duc_thang"]
df_phenikaa = dataframes["tuyen_sinh_phenikaa"]

In [7]:
print(df_tonducthang.head())  # Xem 5 dòng đầu tiên
print(df_phenikaa.info())       # Xem thông tin tổng quan

                                                Text
0                             Đại học,Tin tuyển sinh
1        Đại học (https://tuyensinh.edu.vn/dai-hoc/)
2  Tin tuyển sinh (https://tuyensinh.edu.vn/tin-t...
3  Trường ĐH Tôn Đức Thắng chỉ dành 25% chỉ tiêu ...
4  Theo phương án tuyển sinh dự kiến, năm 2022 Tr...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    79 non-null     object
dtypes: object(1)
memory usage: 764.0+ bytes
None


In [8]:
import re
import os
import spacy
from collections import Counter

In [9]:
def clean_text(text):
    """Xóa dấu câu (trừ . ! ?), giữ chữ thường"""
    text = re.sub(r"[^\w\s.!?]", "", text)  # Giữ lại dấu chấm, chấm than, chấm hỏi
    return text.strip()

# Tạo bản sao DataFrame để làm sạch dữ liệu
df_phenikaa_cleaned = df_phenikaa.copy()
df_tonducthang_cleaned = df_tonducthang.copy()

df_phenikaa_cleaned["Text"] = df_phenikaa_cleaned["Text"].apply(clean_text)
df_tonducthang_cleaned["Text"] = df_tonducthang_cleaned["Text"].apply(clean_text)

# Kiểm tra dữ liệu sau khi làm sạch
print(df_phenikaa_cleaned.head(), df_tonducthang_cleaned.head())

# Kiểm tra dữ liệu sau làm sạch
df_phenikaa.head(), df_tonducthang.head()

                                                Text
0                              Đại họcTin tuyển sinh
1                Đại học httpstuyensinh.edu.vndaihoc
2   Tin tuyển sinh httpstuyensinh.edu.vntintuyensinh
3  Trường Đại học Phenikaa công bố 5 phương thức ...
4  Trường Đại học Phenikaa công bố thông tin tuyể...                                                 Text
0                              Đại họcTin tuyển sinh
1                Đại học httpstuyensinh.edu.vndaihoc
2   Tin tuyển sinh httpstuyensinh.edu.vntintuyensinh
3  Trường ĐH Tôn Đức Thắng chỉ dành 25 chỉ tiêu x...
4  Theo phương án tuyển sinh dự kiến năm 2022 Trư...


(                                                Text
 0                             Đại học,Tin tuyển sinh
 1        Đại học (https://tuyensinh.edu.vn/dai-hoc/)
 2  Tin tuyển sinh (https://tuyensinh.edu.vn/tin-t...
 3  Trường Đại học Phenikaa công bố 5 phương thức ...
 4  Trường Đại học Phenikaa công bố thông tin tuyể...,
                                                 Text
 0                             Đại học,Tin tuyển sinh
 1        Đại học (https://tuyensinh.edu.vn/dai-hoc/)
 2  Tin tuyển sinh (https://tuyensinh.edu.vn/tin-t...
 3  Trường ĐH Tôn Đức Thắng chỉ dành 25% chỉ tiêu ...
 4  Theo phương án tuyển sinh dự kiến, năm 2022 Tr...)

In [10]:
# Load Vietnamese NLP model
nlp = spacy.blank("vi")
nlp.add_pipe("sentencizer")

In [11]:
def split_sentences(text):
    """Chia văn bản thành câu dựa trên dấu câu bằng spaCy"""
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

def count_words(text):
    """Đếm số từ bằng spaCy"""
    doc = nlp(text)
    return len([token for token in doc if token.is_alpha])

def count_stats(texts):
    """Đếm tổng số từ, câu và đoạn văn """
    total_words = sum(count_words(text) for text in texts)
    total_sentences = sum(len(split_sentences(text)) for text in texts)
    total_paragraphs = sum(len(text.strip().split("\n")) for text in texts)

    return {
        "Total Words": total_words,
        "Total Sentences": total_sentences,
        "Total Paragraphs": total_paragraphs
    }

# Thống kê cho từng tài liệu
stats_phenikaa = count_stats(df_phenikaa_cleaned["Text"])
stats_tonducthang = count_stats(df_tonducthang_cleaned["Text"])

stats_phenikaa, stats_tonducthang

({'Total Words': 816, 'Total Sentences': 153, 'Total Paragraphs': 79},
 {'Total Words': 285, 'Total Sentences': 57, 'Total Paragraphs': 27})

In [12]:
def compute_word_frequency(texts):
    """Tạo danh sách từ vựng ban đầu và tính tần suất"""
    vocab = Counter()
    for text in texts:
        words = text.split()
        vocab.update(words)
    return vocab

# Tính từ vựng ban đầu
vocab_phenikaa = compute_word_frequency(df_phenikaa_cleaned["Text"])
vocab_tonducthang = compute_word_frequency(df_tonducthang_cleaned["Text"])

# Hiển thị 10 từ phổ biến nhất
vocab_phenikaa.most_common(10), vocab_tonducthang.most_common(10)

([('tuyển', 62),
  ('học', 61),
  ('xét', 48),
  ('sinh', 40),
  ('các', 28),
  ('có', 23),
  ('điểm', 23),
  ('với', 22),
  ('từ', 21),
  ('Trường', 20)],
 [('tuyển', 21),
  ('học', 19),
  ('xét', 18),
  ('chỉ', 16),
  ('sinh', 14),
  ('ĐH', 13),
  ('trình', 12),
  ('THPT', 11),
  ('chương', 11),
  ('kết', 11)])

In [13]:
STOPWORDS_FILE = "vietnamese-stopwords.txt"
VIETNAMESE_STOPWORDS = set()

if os.path.exists(STOPWORDS_FILE):
    with open(STOPWORDS_FILE, "r", encoding="utf-8") as file:
        VIETNAMESE_STOPWORDS = set(file.read().strip().split("\n"))

def remove_stopwords(vocab):
    """Loại bỏ stopwords khỏi từ vựng"""
    return {word: freq for word, freq in vocab.items() if word not in VIETNAMESE_STOPWORDS}

# Tạo từ vựng mới sau khi loại bỏ stopwords
vocab_v2_phenikaa = remove_stopwords(vocab_phenikaa)
vocab_v2_tonducthang = remove_stopwords(vocab_tonducthang)

# Hiển thị 10 từ phổ biến nhất sau khi loại bỏ stopwords
sorted(vocab_v2_phenikaa.items(), key=lambda x: x[1], reverse=True)[:10], sorted(vocab_v2_tonducthang.items(), key=lambda x: x[1], reverse=True)[:10]

([('tuyển', 62),
  ('học', 61),
  ('xét', 48),
  ('sinh', 40),
  ('Trường', 20),
  ('hợp', 20),
  ('Đại', 18),
  ('thí', 17),
  ('môn', 15),
  ('trở', 15)],
 [('tuyển', 21),
  ('học', 19),
  ('xét', 18),
  ('sinh', 14),
  ('ĐH', 13),
  ('trình', 12),
  ('THPT', 11),
  ('chương', 11),
  ('kết', 11),
  ('Trường', 10)])

In [14]:
def extract_ngrams(texts):
    """Trích xuất unigrams, bigrams và trigrams"""
    unigrams, bigrams, trigrams = [], [], []
    
    for text in texts:
        words = text.split()
        unigrams.extend(words)
        bigrams.extend(zip(words[:-1], words[1:]))
        trigrams.extend(zip(words[:-2], words[1:-1], words[2:]))

    return unigrams, bigrams, trigrams

# Trích xuất n-grams
unigrams_p, bigrams_p, trigrams_p = extract_ngrams(df_phenikaa_cleaned["Text"])
unigrams_t, bigrams_t, trigrams_t = extract_ngrams(df_tonducthang_cleaned["Text"])

# Hiển thị kết quả
len(unigrams_p), len(bigrams_p), len(trigrams_p), len(unigrams_t), len(bigrams_t), len(trigrams_t)

(1816, 1737, 1662, 605, 578, 554)

In [26]:
def extract_anchor_texts(texts):
    """Trích xuất anchor text từ danh sách văn bản có dạng text (link)"""
    anchor_texts = []
    
    # Regex tìm text có dạng "text (URL)"
    pattern = r"(\S.*?)(?:\s*)\((https?://[^\s)]+)\)"

    # Chuyển đổi pandas.Series thành danh sách chuỗi
    if isinstance(texts, pd.Series):
        texts = texts.dropna().astype(str).tolist()

    # Duyệt từng chuỗi trong danh sách
    for text in texts:
        matches = re.findall(pattern, text)
        anchor_texts.extend([match[0].strip() for match in matches])

    return anchor_texts

df_phenikaa_anchor = df_phenikaa.copy()
df_tonducthang_anchor = df_tonducthang.copy()

df_phenikaa_anchor["Text"] = df_phenikaa_anchor["Text"].apply(lambda x: extract_anchor_texts([x]) if pd.notna(x) else [])
df_tonducthang_anchor["Text"] = df_tonducthang_anchor["Text"].apply(lambda x: extract_anchor_texts([x]) if pd.notna(x) else [])

df_phenikaa_anchor = [x for x in df_phenikaa_anchor["Text"].tolist() if x]  
df_tonducthang_anchor = [x for x in df_tonducthang_anchor["Text"].tolist() if x] 

print("Phenikaa:", df_phenikaa_anchor)
print("Ton Duc Thang:", df_tonducthang_anchor)

Phenikaa: [['Đại học'], ['Tin tuyển sinh'], ['Điểm chuẩn Đại Học Giáo Dục – Đại học Quốc Gia Hà Nội 2021'], ['ĐH Công nghiệp Hà Nội thêm 3 phương thức xét tuyển mới']]
Ton Duc Thang: [['Đại học'], ['Tin tuyển sinh'], ['tại đây'], ['Tuyển sinh 2022: Nhóm ngành ‘nóng’ có nhiều biến động'], ['Năm 2022, ĐH Quốc gia TP.HCM tổ chức 2 đợt thi đánh giá năng lực vào tháng 3 và 5']]


In [30]:
import spacy

# Load mô hình NER tiếng Việt (chọn một trong hai)
nlp2 = spacy.load("vi_core_news_md")  # hoặc "vi_core_news_md"

def extract_named_entities(texts):
    """Thực hiện Named Entity Recognition (NER)"""
    entities = []
    for text in texts.dropna():  # Bỏ các giá trị NaN trước khi xử lý
        doc = nlp2(text)
        for ent in doc.ents:
            entities.append((ent.text, ent.label_))
    return entities

# Thực hiện NER
ner_phenikaa = extract_named_entities(df_phenikaa["Text"])
ner_tonducthang = extract_named_entities(df_tonducthang["Text"])

# In 10 kết quả đầu tiên
print("Phenikaa NER:", ner_phenikaa[:10])
print("Ton Duc Thang NER:", ner_tonducthang[:10])

OSError: [E050] Can't find model 'vi_core_news_md'. It doesn't seem to be a Python package or a valid path to a data directory.